In [ ]:
import fasttext
import pandas as pd
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
archive = zipfile.ZipFile('../input/sentiment-analysis-on-movie-reviews/train.tsv.zip', 'r')
train_df = pd.read_csv(archive.open('train.tsv'), sep='\t')

archive = zipfile.ZipFile('../input/sentiment-analysis-on-movie-reviews/test.tsv.zip', 'r')
test_df = pd.read_csv(archive.open('test.tsv'), sep='\t')

In [ ]:
train_df, valid_df = train_test_split(train_df, test_size=0.01)

In [ ]:
with open('ft_train.txt', mode='w') as f:
    for i in train_df.to_dict(orient='records'):
        print(f'__label__{i["Sentiment"]} {i["Phrase"]}', file=f)

In [ ]:
! head -n 5 ft_train.txt

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.vec.gz
!gzip -d cc.en.300.vec.gz
!ls -alh

In [ ]:
model = fasttext.train_supervised('ft_train.txt', epoch=64, lr=0.1, dim=300, pretrainedVectors='cc.en.300.vec')

In [ ]:
y_true_list = []
y_pred_list = []
for i in valid_df.to_dict(orient='records'):
    y_pred = model.predict(i['Phrase'])
    y_pred = int(str(y_pred[0][0])[-1])
    y_true = i['Sentiment']
    y_true_list.append(y_true)
    y_pred_list.append(y_pred)

In [ ]:
print(classification_report(y_true_list, y_pred_list))

In [ ]:
submit_df = []
for i in test_df.to_dict(orient='records'):
    y_pred = model.predict(i['Phrase'])
    y_pred = int(str(y_pred[0][0])[-1])
    submit_df.append({
        'PhraseId': i['PhraseId'],
        'Sentiment': y_pred
    })

In [ ]:
submit_df = pd.DataFrame(submit_df)
submit_df.to_csv('submission.csv', index=False)
submit_df